In [ ]:
# !pip install -q -U trl accelerate git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git
# !pip install -q datasets bitsandbytes wandb
# !pip install -q datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 90.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 24.0 MB/s et

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import json
pd.set_option('display.max_colwidth', None)

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer, AutoModelForQuestionAnswering
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model,AutoPeftModelForCausalLM
import os,torch, wandb, platform, warnings
from datasets import load_dataset
from trl import SFTTrainer
from huggingface_hub import notebook_login


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /opt/conda/envs/mar39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/opt/conda/envs/mar39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: /opt/conda did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)


In [ ]:
from langchain.document_loaders import AsyncChromiumLoader, HuggingFaceDatasetLoader, DataFrameLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
# from langchain.chains import RunnablePassthrough, llm_chain
from langchain.chains import LLMChain, RetrievalQA
from langchain.schema.runnable import RunnablePassthrough
import nest_asyncio
from langchain.document_transformers import Html2TextTransformer

from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

### Data Preprocessing

In [4]:
with open('train_webmd_squad_v2_consec.json', 'r') as json_file:
    data = json.load(json_file)

In [3]:
def extract_question_context_answer_triples(data):
    question_context_answer_triples = []

    for item in data['data']:
        for paragraph in item['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                answers = [answer_info['text'] for answer_info in qa['answers']]
                for answer in answers:
                    question_context_answer_triples.append((question, context, answer))

    return question_context_answer_triples

In [6]:
question_context_answer_triples = extract_question_context_answer_triples(data)

In [7]:
df = pd.DataFrame(question_context_answer_triples, columns=['Question', 'Context', 'Answer'])

In [8]:
df.head()

,Question,Context,Answer
0,What surgical techniques are used to treat gla...,Treatment of open-angle glaucoma -- the most c...,If the glaucoma does not respond to medication...
1,What are the best ways to treat glaucoma?,Treatment of open-angle glaucoma -- the most c...,Both drugs and surgery have high rates of succ...
2,What should you know about treating open-angle...,Treatment of open-angle glaucoma -- the most c...,That is why it's so important to have your eye...
3,Is surgery for glaucoma dangerous?,Treatment of open-angle glaucoma -- the most c...,"Before giving your consent, always ask the sur..."
4,How is acute closed-angle glaucoma treated?,Treatment of open-angle glaucoma -- the most c...,Acute angle-closure glaucoma is different from...


In [9]:
df.shape

(19989, 3)

### Load the Model

In [14]:
model = "mistralai/Mistral-7B-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

base_model = AutoModelForCausalLM.from_pretrained(
    model,
    quantization_config=bnb_config,
    # use_cache=False,
    trust_remote_code=True
)
base_model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Prompt Engineering

#### **1. Zero-shot**

In [15]:
prompt1 = "What types of exercise are best for people with asthma?"
prompt2 = "How is obsessive-compulsive disorder diagnosed?"
prompt3 = "When are you more likely to get a blood clot?"
prompt4 = "How should you lift objects to prevent back pain?"
prompt5 = "How can you be smart with antibiotics?"

In [16]:
device = base_model.device

inputs = tokenizer(prompt1, return_tensors="pt").to(device)
outputs = base_model.generate(**inputs, max_length=250, top_p=0.9, temperature=0.1)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/opt/conda/envs/mar39/lib/python3.9/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/opt/conda/envs/mar39/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/envs/mar39/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based ge

What types of exercise are best for people with asthma?

Asthma is a chronic lung disease that causes inflammation and narrowing of the airways. It can cause shortness of breath, wheezing, coughing, and chest tightness.

Exercise can be a trigger for asthma symptoms. But it can also help improve lung function and reduce the frequency of asthma attacks.

The American College of Sports Medicine (ACSM) recommends that people with asthma exercise at least 30 minutes a day, five days a week.

The ACSM also recommends that people with asthma exercise at a moderate intensity. This means that you should be able to talk, but not sing, while exercising.

There are many different types of exercise that can be beneficial for people with asthma. Some of the most popular types of exercise include:

- Walking
- Swimming
- Cycling
- Yoga
- Pilates
- Tai chi

Each of these types of exercise has its own benefits for people with asthma.

Walking is a low-impact exercise


In [17]:
device = base_model.device

inputs = tokenizer(prompt2, return_tensors="pt").to(device)
outputs = base_model.generate(**inputs, max_length=250, top_p=0.9, temperature=0.1)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


How is obsessive-compulsive disorder diagnosed?

Obsessive-compulsive disorder is diagnosed by a mental health professional, such as a psychiatrist or psychologist. The professional will conduct a clinical interview and may ask you to fill out a questionnaire.

The Diagnostic and Statistical Manual of Mental Disorders, Fifth Edition (DSM-5), published by the American Psychiatric Association, is the standard classification of mental disorders used by mental health professionals in the United States. The DSM-5 criteria for obsessive-compulsive disorder include:

- Obsessions or compulsions that are time-consuming (more than an hour a day) or cause significant distress or impairment in social, occupational, or other important areas of functioning.
- The obsessions or compulsions are not due to the direct physiological effects of a substance (e.g., a drug of abuse, a medication) or a general medical condition (e.g., hyperthyroidism).

The DSM-5 also includes criteria for obsessive-compulsi

In [18]:
device = base_model.device

inputs = tokenizer(prompt3, return_tensors="pt").to(device)
outputs = base_model.generate(**inputs, max_length=250, top_p=0.9, temperature=0.1)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


When are you more likely to get a blood clot?

- When you are pregnant
- When you are taking birth control pills
- When you are taking hormone replacement therapy
- All of the above

Answer: All of the above.

Blood clots are a serious health concern for women. They can cause heart attacks, strokes, and pulmonary embolisms.

The risk of blood clots is higher during pregnancy and when taking birth control pills or hormone replacement therapy.

If you are pregnant, taking birth control pills, or taking hormone replacement therapy, you should be aware of the signs and symptoms of blood clots.

If you experience any of the following symptoms, you should seek medical attention immediately:

- Chest pain
- Shortness of breath
- Coughing up blood
- Swelling in the legs
- Pain in the legs
- Redness or warmth in the legs

If you are pregnant, taking birth control pills, or taking hormone replacement therapy, you should be aware of the signs and symptoms of blood clots.

If you experience any of

In [ ]:
device = base_model.device

inputs = tokenizer(prompt4, return_tensors="pt").to(device)
outputs = base_model.generate(**inputs, max_length=250, top_p=0.9, temperature=0.1)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


How should you lift objects to prevent back pain?

The best way to lift an object is to bend your knees and keep your back straight.

This is called the “squat” position.

When you lift an object, you should:

- Bend your knees and keep your back straight.
- Keep the object close to your body.
- Lift with your legs.
- Avoid twisting your body.

If you have to lift an object that is heavy or awkward, get help from someone else.

If you have to lift an object that is heavy or awkward, get help from someone else.

If you have to lift an object that is heavy or awkward, get help from someone else.

If you have to lift an object that is heavy or awkward, get help from someone else.

If you have to lift an object that is heavy or awkward, get help from someone else.

If you have to lift an object that is heavy or awkward, get help from someone else.

If you have to lift an object that is heavy or awkward, get help from someone else.

If you have to lift an object


In [ ]:
device = base_model.device

inputs = tokenizer(prompt5, return_tensors="pt").to(device)
outputs = base_model.generate(**inputs, max_length=250, top_p=0.9, temperature=0.1)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


How can you be smart with antibiotics?

Antibiotics are medicines that are used to treat infections caused by bacteria. They do not work against viruses, such as those that cause the common cold or flu.

Antibiotics are not always needed. Most coughs, colds, sore throats, and other minor illnesses will get better on their own without antibiotics.

Antibiotics are not always the answer. Antibiotics are not always needed to treat an infection.

Antibiotics are not always effective. Antibiotics are not always effective against bacteria.

Antibiotics are not always safe. Antibiotics can cause side effects, such as diarrhea, nausea, and rash.

Antibiotics are not always necessary. Antibiotics are not always necessary to treat an infection.

Antibiotics are not always the answer. Antibiotics are not always the answer to every infection.

Antibiotics are not always effective. Antibiotics are not always effective against bacteria.

Antibiotics are not always


#### **2. Few-shot**

In [ ]:
# Example questions and answers for few-shot learning
few_shot_examples = [("What surgical techniques are used to treat glaucoma?", "If the glaucoma does not respond to medication, or if you cannot tolerate the side effects, your doctor may change medications or recommend one of several surgical techniques: Laser trabeculoplasty creates small laser burns in the area where the fluid drains, improving the outflow rate of aqueous fluid. This relatively brief procedure can often be done in an ophthalmologist's clinic. Trabeculectomy is a surgical procedure that creates a new channel for fluid outflow in cases in which the intraocular pressure is high and the optic nerve damage progresses. Long-term results vary, but generally, the success rate is good. Surgical implants that shunt fluid out of the eye may also be used to decrease pressure in the eye. Remember, all forms of medical or surgical treatment have potential benefits and risks. Before giving your consent, always ask the surgeon to clearly explain any treatment or surgery as well as the proposed benefits, effective alternatives, and potential risks or complications."),
    ("What are the best ways to treat glaucoma?","Both drugs and surgery have high rates of success in treating chronic open-angle glaucoma, but you can help yourself by carefully following the doctor's treatment plan. Some patients may find it difficult to follow a regimen involving two or three different eye drops. Be candid and tell the doctor if you cannot follow the medication schedule or if the eye drops cause unwanted side effects. There are frequently alternative treatments. Because of potential drug interactions, be sure to tell your doctor about any other medical problems you have or other medications you take. If glaucoma drops causes the eyes to become chronically red, consult your doctor about switching to preservative-free glaucoma drops that may alleviate the redness from preservatives."),
    ("Is surgery for glaucoma dangerous?","Before giving your consent, always ask the surgeon to clearly explain any treatment or surgery as well as the proposed benefits, effective alternatives, and potential risks or complications."),
    ("How is acute closed-angle glaucoma treated?","Acute angle-closure glaucoma is different from chronic open-angle glaucoma in several important ways: The symptoms usually occur with relative suddenness; the eye is painful and red. If the high pressure in the eye is not relieved quickly, blindness can occur. On the other hand, treatments for acute angle-closure glaucoma -- usually laser treatment -- are permanent and do not require long-term therapy. For this type of glaucoma, making a hole in the iris to allow fluid to drain, called an iridectomy, is the standard treatment to cure it. The unaffected eye also is usually treated to prevent a future attack. However, it's important to get your eyes checked regularly, as some people may develop a case of chronic angle-closure glaucoma later in life, even after laser treatment."),
    ("What should you know about treating open-angle glaucoma?","That is why it's so important to have your eye doctor test you regularly for glaucoma. Once diagnosed, glaucoma is usually controlled with eye drops that reduce eye pressure. Glaucoma is a life-long condition and needs continual follow-up with your eye doctor. Both drugs and surgery have high rates of success in treating chronic open-angle glaucoma, but you can help yourself by carefully following the doctor's treatment plan. Some patients may find it difficult to follow a regimen involving two or three different eye drops. Be candid and tell the doctor if you cannot follow the medication schedule or if the eye drops cause unwanted side effects. There are frequently alternative treatments. Because of potential drug interactions, be sure to tell your doctor about any other medical problems you have or other medications you take. If glaucoma drops causes the eyes to become chronically red, consult your doctor about switching to preservative-free glaucoma drops that may alleviate the redness from preservatives. Acute angle-closure glaucoma is different from chronic open-angle glaucoma in several important ways: The symptoms usually occur with relative suddenness; the eye is painful and red. If the high pressure in the eye is not relieved quickly, blindness can occur."),
    ]

In [20]:
# New question
new_question = prompt1

# Constructing the few-shot prompt with an explicit instruction
prompt = "\n\n".join([f"Q: {q}\nA: {a}" for q, a in few_shot_examples])
# prompt += "\n\nInstruction: Follow the pattern of the example questions and answers to answer the new question in 150  words."
prompt += f"\n\nQ: {new_question}\nA: "

# Tokenize and generate the response
device = base_model.device
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Adjusting max_length if needed
max_length_adjusted = min(len(inputs['input_ids'][0]) + 250, tokenizer.model_max_length)

outputs = base_model.generate(**inputs, max_length=max_length_adjusted, top_p=0.9, temperature=0.1, pad_token_id=tokenizer.eos_token_id)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extracting only the answer part
answer_start = response.find(f"Q: {new_question}\nA: ") + len(f"Q: {new_question}\nA: ")
answer = response[answer_start:].split("\n")[0].strip()

# Print the model's answer
print("\nModel's Answer:\n")
print(answer)



Model's Answer:

1. Swimming. Swimming is a great exercise for people with asthma. The humidity in the air and the warm temperature of the water can help to open up your airways. 2. Walking. Walking is a great exercise for people with asthma. It is low impact and can be done at your own pace. 3. Yoga. Yoga is a great exercise for people with asthma. The deep breathing exercises can help to open up your airways and the stretching can help to improve your flexibility. 4. Cycling. Cycling is a great exercise for people with asthma. It is low impact and can be done at your own pace. 5. Pilates. Pilates is a great exercise for people with asthma. The deep breathing exercises can help to open up your airways and the stretching can help to improve your flexibility. 6. Tai Chi. Tai Chi is a great exercise for people with asthma. The deep breathing exercises can help to open up your airways and the stretching can help to improve your flexibility. 7. Dancing. Dancing is a great exercise for peo

In [21]:
new_question = prompt2

# Constructing the few-shot prompt with an explicit instruction
prompt = "\n\n".join([f"Q: {q}\nA: {a}" for q, a in few_shot_examples])
# prompt += "\n\nInstruction: Follow the pattern of the example questions and answers to answer the new question in 150  words."
prompt += f"\n\nQ: {new_question}\nA: "

# Tokenize and generate the response
device = base_model.device
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Adjusting max_length if needed
max_length_adjusted = min(len(inputs['input_ids'][0]) + 250, tokenizer.model_max_length)

outputs = base_model.generate(**inputs, max_length=max_length_adjusted, top_p=0.9, temperature=0.1, pad_token_id=tokenizer.eos_token_id)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extracting only the answer part
answer_start = response.find(f"Q: {new_question}\nA: ") + len(f"Q: {new_question}\nA: ")
answer = response[answer_start:].split("\n")[0].strip()

# Print the model's answer
print("\nModel's Answer:\n")
print(answer)


Model's Answer:

1. The patient has obsessions, compulsions, or both. 2. The obsessions or compulsions are time-consuming (more than 1 hour a day) or cause marked distress or significant impairment in social, occupational, or other important areas of functioning. 3. The obsessions or compulsions are not due to the direct physiological effects of a substance (e.g., a drug of abuse, a medication) or a general medical condition (e.g., hyperthyroidism). 4. The obsessions or compulsions are not better accounted for by another mental disorder (e.g., anxiety disorder, body dysmorphic disorder, anorexia nervosa, obsessive-compulsive personality disorder).


In [22]:
new_question = prompt3

# Constructing the few-shot prompt with an explicit instruction
prompt = "\n\n".join([f"Q: {q}\nA: {a}" for q, a in few_shot_examples])
# prompt += "\n\nInstruction: Follow the pattern of the example questions and answers to answer the new question in 150  words."
prompt += f"\n\nQ: {new_question}\nA: "

# Tokenize and generate the response
device = base_model.device
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Adjusting max_length if needed
max_length_adjusted = min(len(inputs['input_ids'][0]) + 250, tokenizer.model_max_length)

outputs = base_model.generate(**inputs, max_length=max_length_adjusted, top_p=0.9, temperature=0.1, pad_token_id=tokenizer.eos_token_id)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extracting only the answer part
answer_start = response.find(f"Q: {new_question}\nA: ") + len(f"Q: {new_question}\nA: ")
answer = response[answer_start:].split("\n")[0].strip()

# Print the model's answer
print("\nModel's Answer:\n")
print(answer)


Model's Answer:

1. If you are over 60 years old. 2. If you are a woman. 3. If you have a family history of blood clots. 4. If you have had a blood clot before. 5. If you have had a heart attack or stroke. 6. If you have had surgery or an injury. 7. If you are pregnant. 8. If you are taking birth control pills. 9. If you are taking estrogen. 10. If you are taking certain medicines. 11. If you are overweight. 12. If you smoke. 13. If you are not active. 14. If you have a blood clotting disorder. 15. If you have a catheter in a vein. 16. If you have cancer. 17. If you have a central venous catheter. 18. If you have a pacemaker. 19. If you have a stent. 20. If you have a port-a-cath. 21. If you have a central venous catheter.


In [23]:
new_question = prompt4

# Constructing the few-shot prompt with an explicit instruction
prompt = "\n\n".join([f"Q: {q}\nA: {a}" for q, a in few_shot_examples])
# prompt += "\n\nInstruction: Follow the pattern of the example questions and answers to answer the new question in 150  words."
prompt += f"\n\nQ: {new_question}\nA: "

# Tokenize and generate the response
device = base_model.device
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Adjusting max_length if needed
max_length_adjusted = min(len(inputs['input_ids'][0]) + 250, tokenizer.model_max_length)

outputs = base_model.generate(**inputs, max_length=max_length_adjusted, top_p=0.9, temperature=0.1, pad_token_id=tokenizer.eos_token_id)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extracting only the answer part
answer_start = response.find(f"Q: {new_question}\nA: ") + len(f"Q: {new_question}\nA: ")
answer = response[answer_start:].split("\n")[0].strip()

# Print the model's answer
print("\nModel's Answer:\n")
print(answer)


Model's Answer:

1. Bend your knees and keep your back straight. 2. Lift with your legs, not your back. 3. Keep the object close to your body. 4. Avoid twisting your body. 5. Do not lift heavy objects. 6. Do not lift and hold objects for long periods. 7. Do not lift and carry objects above shoulder level. 8. Do not lift and carry objects below knee level. 9. Do not lift and carry objects that are too heavy for you. 10. Do not lift and carry objects that are too heavy for you. 11. Do not lift and carry objects that are too heavy for you. 12. Do not lift and carry objects that are too heavy for you. 13. Do not lift and carry objects that are too heavy for you. 14. Do not lift and carry objects that are too heavy for you. 15. Do not lift and carry objects that are too heavy for you. 16. Do not lift and carry objects that are too heavy for you. 17. Do not lift and carry objects that are too heavy for you. 18. Do


In [24]:
new_question = prompt5

# Constructing the few-shot prompt with an explicit instruction
prompt = "\n\n".join([f"Q: {q}\nA: {a}" for q, a in few_shot_examples])
# prompt += "\n\nInstruction: Follow the pattern of the example questions and answers to answer the new question in 150  words."
prompt += f"\n\nQ: {new_question}\nA: "

# Tokenize and generate the response
device = base_model.device
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Adjusting max_length if needed
max_length_adjusted = min(len(inputs['input_ids'][0]) + 250, tokenizer.model_max_length)

outputs = base_model.generate(**inputs, max_length=max_length_adjusted, top_p=0.9, temperature=0.1, pad_token_id=tokenizer.eos_token_id)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extracting only the answer part
answer_start = response.find(f"Q: {new_question}\nA: ") + len(f"Q: {new_question}\nA: ")
answer = response[answer_start:].split("\n")[0].strip()

# Print the model's answer
print("\nModel's Answer:\n")
print(answer)


Model's Answer:

1. Don't take antibiotics unless prescribed by a doctor. 2. Don't take antibiotics prescribed for someone else. 3. Don't take antibiotics for viral infections, such as colds, flu, and most sore throats. 4. Don't take antibiotics for a long time without a doctor's advice. 5. Don't take antibiotics for a long time without a doctor's advice. 6. Don't take antibiotics for a long time without a doctor's advice. 7. Don't take antibiotics for a long time without a doctor's advice. 8. Don't take antibiotics for a long time without a doctor's advice. 9. Don't take antibiotics for a long time without a doctor's advice. 10. Don't take antibiotics for a long time without a doctor's advice. 11. Don't take antibiotics for a long time without a doctor's advice. 12. Don't take antibiotics for a long time without a doctor'


#### **3. Chain of Thought**

In [25]:
# Function to find the question and context for questions containing 'asthma' but not 'BDD'
def get_question_and_context_for_asthma_excluding_bdd(df):
    # Filter the DataFrame for rows where the 'Question' contains 'asthma' and does not contain 'BDD'
    asthma_rows = df[
        df['Question'].str.contains(r'\bOCD\b', case=False, regex=True) &
        ~df['Question'].str.contains(r'\bBDD\b', case=False, regex=True)
    ]
    
    # If any rows are found, return the 'Question' and 'Context' of the first such row
    if not asthma_rows.empty:
        question = asthma_rows.iloc[0]['Question']
        context = asthma_rows.iloc[0]['Context']
        return question, context
    else:
        return "No question with the keyword 'asthma' found, excluding 'BDD'.", None

# Call the function and print the results
question_with_asthma, context_2 = get_question_and_context_for_asthma_excluding_bdd(df)

# Example question based on the context
question = "How is obsessive-compulsive disorder diagnosed?"

# Constructing the prompt
prompt = f"Context: {context_2}\n\nQuestion: {question}\n\nInstruction: Based on the context, answer the question.\n"

# Tokenize and generate the response
inputs = tokenizer(prompt, return_tensors="pt").to(device)
max_length_adjusted = min(len(inputs['input_ids'][0]) + 250, tokenizer.model_max_length)
outputs = base_model.generate(**inputs, max_length=max_length_adjusted, top_p=0.9, temperature=0.1, pad_token_id=tokenizer.eos_token_id)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the full model's response for debugging
print("\nFull Model's Response:\n", response)

# Extracting only the answer part
answer = response.split('\n')[-1].strip()

# Print only the model's answer
print("\nModel's Answer:\n", answer)


Full Model's Response:
 Context: Does it take you forever to make a doctor's appointment, clean out your garage, or do your taxes? Putting off something that needs to be done is called procrastination. We all do it sometimes. But if you constantly struggle to finish tasks, there may be a bigger problem at play. Once you figure out your reasons, you can work on making the most of your time. When you're facing something you don't want to do, it can make you feel tense. Putting off that dreaded chore is a way to get some short-term relief. The downside is that you'll still have to tackle your task in the future, which may make you feel guilty or angry -- and cause your stress to rise more. If the tension's bad enough to keep you from getting things done, you might notice it affects you in other ways, too, like: Trouble sleeping Racing thoughts Lack of energy or trouble focusing Headaches or muscle tension Keep your mood in check with regular exercise, limit alcohol and caffeine (which ca

In [26]:
# Function to find the question and context for questions containing 'asthma'
def get_question_and_context_for_asthma(df):
    # Filter the DataFrame for rows where the 'Question' column contains the word 'asthma'
    asthma_rows = df[df['Question'].str.contains(r'\bblood clot\b', case=False, regex=True)]
    
    # If any rows are found, return the 'Question' and 'Context' of the first such row
    if not asthma_rows.empty:
        question = asthma_rows.iloc[0]['Question']
        context = asthma_rows.iloc[0]['Context']
        return question, context

# Call the function and print the results
question_with_asthma, context_3 = get_question_and_context_for_asthma(df)

# Example question based on the context
question = "When are you more likely to get a blood clot?"

# Constructing the prompt
prompt = f"Context: {context_3}\n\nQuestion: {question}\n\nInstruction: Based on the context, answer the question.\n"


# Tokenize and generate the response
inputs = tokenizer(prompt, return_tensors="pt").to(device)
max_length_adjusted = min(len(inputs['input_ids'][0]) + 250, tokenizer.model_max_length)
outputs = base_model.generate(**inputs, max_length=max_length_adjusted, top_p=0.9, temperature=0.1, pad_token_id=tokenizer.eos_token_id)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the full model's response for debugging
print("\nFull Model's Response:\n", response)

# Extracting only the answer part
answer = response.split('\n')[-1].strip()

# Print only the model's answer
print("\nModel's Answer:\n", answer)


Full Model's Response:
 Context: After hurting your leg, you're probably dealing with some discomfort and inconvenience. But that's not all you have to be concerned about. This type of injury raises your chances for getting a blood clot. Any time a blood vessel gets damaged, the nearby blood can thicken and organize into a sticky clump, or clot. Some clots only affect veins near your skin's surface. This condition, called superficial thrombophlebitis, typically doesn't lead to serious problems. When a blood clot forms farther inside your leg, it's known as deep vein thrombosis ( DVT). These clots can be dangerous if they break loose and travel to your lungs. Doctors call this a pulmonary embolism ( PE). Trauma could result from a car accident, a sports injury, or even a fall. Common mishaps that may lead to a clot include: Broken bones Bad bumps Severe bruises Severe muscle injuries A 2008 study revealed even minor leg injuries -- ones that don't need a cast or bed rest -- can raise y

In [27]:
# Function to find the question and context for questions containing 'asthma'
def get_question_and_context_for_asthma(df):
    # Filter the DataFrame for rows where the 'Question' column contains the word 'asthma'
    asthma_rows = df[df['Question'].str.contains(r'\bprevent back pain\b', case=False, regex=True)]
    
    # If any rows are found, return the 'Question' and 'Context' of the first such row
    if not asthma_rows.empty:
        question = asthma_rows.iloc[0]['Question']
        context = asthma_rows.iloc[0]['Context']
        return question, context
    else:
        return "No question with the keyword 'asthma' found.", None

# Call the function and print the results
question_with_asthma, context_4 = get_question_and_context_for_asthma(df)

# Example question based on the context
question = "How should you lift objects to prevent back pain?"

# Constructing the prompt
prompt = f"Context: {context_4}\n\nQuestion: {question}\n\nInstruction: Based on the context, answer the question.\n"


# Tokenize and generate the response
inputs = tokenizer(prompt, return_tensors="pt").to(device)
max_length_adjusted = min(len(inputs['input_ids'][0]) + 250, tokenizer.model_max_length)
outputs = base_model.generate(**inputs, max_length=max_length_adjusted, top_p=0.9, temperature=0.1, pad_token_id=tokenizer.eos_token_id)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the full model's response for debugging
print("\nFull Model's Response:\n", response)

# Extracting only the answer part
answer = response.split('\n')[-1].strip()

# Print only the model's answer
print("\nModel's Answer:\n", answer)


Full Model's Response:
 Context: There's a lot riding on your spinal column. It's your body's main structural support. It needs to keep you stable enough to stand upright but flexible enough for movement. So it's no surprise that many people have back problems from time to time. The hurt can stem from sore muscles, ligaments, and tendons, or from herniated disks, fractures, and other problems in your upper, middle, and lower back. Sometimes you feel the effects right away. But in many cases, back problems develop over time. We often bring on our back problems through bad habits, such as: Poor posture, like sitting incorrectly at a desk or behind the steering wheel Repeating the same motion or overdoing it Pushing, pulling, and lifting things carelessly The spine is actually a stack of 24 bones called vertebrae. A healthy spine is S-shaped when viewed from the side. It curves back at your shoulders and inward at your neck and small of your back. It houses and protects your spinal cord,

In [28]:
def get_context_for_specific_question(df, question):
    # Filter the DataFrame for the specific question
    filtered_df = df[df['Question'] == question]

    # Check if there are any matching rows
    if not filtered_df.empty:
        # Return the context of the first matching row
        return filtered_df.iloc[0]['Context']
    else:
        return "No context found for the specified question."

# Call the function with the specific question
specific_question = "Do most people know that antibiotics are not effective for colds or flu?"
context_5 = get_context_for_specific_question(df, specific_question)

# Example question based on the context
question = "How can you be smart with antibiotics?"

# Constructing the prompt
prompt = f"Context: {context_5}\n\nQuestion: {question}\n\nInstruction: Based on the context, answer the question.\n"


# Tokenize and generate the response
inputs = tokenizer(prompt, return_tensors="pt").to(device)
max_length_adjusted = min(len(inputs['input_ids'][0]) + 250, tokenizer.model_max_length)
outputs = base_model.generate(**inputs, max_length=max_length_adjusted, top_p=0.9, temperature=0.1, pad_token_id=tokenizer.eos_token_id)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the full model's response for debugging
print("\nFull Model's Response:\n", response)

# Extracting only the answer part
answer = response.split('\n')[-1].strip()

# Print only the model's answer
print("\nModel's Answer:\n", answer)


Full Model's Response:
 Context: Looking for an effective flu treatment and wondering if antibiotics will work? Antibiotics are medications that fight infections caused by bacteria, but the flu is cause by a virus. Taking antibiotics when you have a virus may do more harm than good. Taking antibiotics when they are not needed increases your risk of getting an infection later that may resist antibiotic treatment. Antibiotics only cure certain infections due to bacteria -- and if taken carelessly, you may get more serious health problems than you bargained for. With any illness, it is critical to address the underlying cause, whether it's bacterial or viral. Antibiotics will not kill cold or flu viruses. Not at all. Antibiotics can save people's lives, and if you need them, you should get them as quickly as you can. Since only a doctor can prescribe antibiotics, this means that you should talk to your doctor if you think you might need them (as opposed to taking your friend's leftover a

In [29]:
import pandas as pd

# Assuming df is your DataFrame already loaded with data

def get_context_for_specific_question(df, question):
    # Filter the DataFrame for the specific question
    filtered_df = df[df['Question'] == question]

    # Check if there are any matching rows
    if not filtered_df.empty:
        # Return the context of the first matching row
        return filtered_df.iloc[0]['Context']
    else:
        return "No context found for the specified question."

# Call the function with the specific question
specific_question = "Is exercise OK if you have asthma?"
context_1 = get_context_for_specific_question(df, specific_question)


# Example question based on the context
question = "What types of exercise are best for people with asthma?"

# Constructing the prompt
prompt = f"Context: {context_1}\n\nQuestion: {question}\n\nInstruction: Based on the context, answer the question.\n"


# Tokenize and generate the response
inputs = tokenizer(prompt, return_tensors="pt").to(device)
max_length_adjusted = min(len(inputs['input_ids'][0]) + 250, tokenizer.model_max_length)
outputs = base_model.generate(**inputs, max_length=max_length_adjusted, top_p=0.9, temperature=0.1, pad_token_id=tokenizer.eos_token_id)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the full model's response for debugging
print("\nFull Model's Response:\n", response)

# Extracting only the answer part
answer = response.split('\n')[-1].strip()

# Print only the model's answer
print("\nModel's Answer:\n", answer)



Full Model's Response:
 Context: There are different types of inhalers that serve different purposes and require different techniques. These inhalers help prevent flares and keep symptoms from getting worse. They're called control inhalers because they have medicine that controls inflammation. Use yours as often as your doctor tells you to, usually once or twice a day: Whether or not you're having symptoms Even if you feel like you're doing better If you're supposed to use it two times a day, aim for 12 hours apart. When you begin using this kind of inhaler, it may be 2 to 4 weeks before you notice the drugs start to work. Rescue or relief inhalers quickly bring back normal breathing when you are: Short of breath Wheezing Feeling tight in your chest Coughing You should keep a rescue inhaler with you all the time. Use it: When you have a flare of symptoms Before you're going to be around your asthma triggers When you run into unexpected triggers A rescue inhaler is for short-term sympt

#### **4. RAG**

##### Create Vector Database

In [8]:
loader = DataFrameLoader(mashqa_df, page_content_column="Context")
# loader = DataFrameLoader(mashqa_df)

In [9]:
docs = loader.load()
# # Use lazy load for larger table, which won't read the full table into memory
# docs = loader.lazy_load()

In [10]:
# Chunk text
text_splitter = CharacterTextSplitter(chunk_size=100, 
                                      chunk_overlap=0)
chunked_documents = text_splitter.split_documents(docs)

In [18]:
# chunked_documents[0]

In [11]:
# Load chunked documents into the FAISS index
db = FAISS.from_documents(chunked_documents, 
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

In [12]:
# Connect query to FAISS index using a retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

In [13]:
query = "What did Laporta say?"
docs = db.similarity_search(query)
# print(docs[0].page_content)
print(docs[0].metadata['Answer'])

One component of red yeast rice, monacolin K, is similar to the active ingredient of some of the cholesterol lowering drugs called statins. Red yeast rice does not seem to be as effective as most conventional statin drugs, however. But these supplements may be a good choice for people who could benefit from only a slight lowering of cholesterol.


In [14]:
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

##### Create LLM Chain

In [15]:
# text_generation_pipeline = transformers.pipeline(
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    #return_full_text=True,
    max_new_tokens=300,
)

prompt_template = """
### [INST] 
Instruction: Write a response to answer the questionby taking help from below context:

{context}

### QUESTION:
{question} 

[/INST]
 """

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt from prompt template 
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain 
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

##### Create RAG Chain

In [24]:
torch.cuda.empty_cache()

In [17]:
query = "What types of exercise are best for people with asthma?" 

retriever = db.as_retriever()

rag_chain = ( 
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

# Invoke the model and capture the response
response = rag_chain.invoke(query)

# Extracting the answer
answer = response['text'].strip()

# Print only the model's answer
print("\nModel's Answer:\n")
print(answer)

/opt/conda/envs/pp/lib/python3.10/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/opt/conda/envs/pp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Model's Answer:

There are several types of exercises that can be beneficial for people with asthma. Low-impact exercises such as swimming, cycling, and walking are generally recommended as they are less likely to trigger asthma symptoms. Yoga and tai chi are also good options as they promote relaxation and can help improve breathing techniques. Strength training exercises can also be helpful in improving overall fitness and reducing the risk of asthma exacerbations. However, it is important for individuals with asthma to consult with their healthcare provider before starting any new exercise program to ensure that it is safe and appropriate for their individual needs and limitations.


In [19]:
query = "How is obsessive-compulsive disorder diagnosed?"

response = rag_chain.invoke(query)
answer = response['text'].strip()
print("\nModel's Answer:\n")
print(answer)

/opt/conda/envs/pp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Model's Answer:

Obsessive-compulsive disorder (OCD) is typically diagnosed through a combination of self-reporting, clinical interview, and psychological testing. The diagnostic criteria for OCD are established by the Diagnostic and Statistical Manual of Mental Disorders (DSM), which is a widely accepted classification system used by mental health professionals to diagnose and treat mental health disorders.

To diagnose OCD, a mental health professional will typically conduct a comprehensive evaluation that includes gathering information about the individual's symptoms, medical history, and family history. The evaluation may also involve administering psychological tests, such as the Yale-Brown Obsessive Compulsive Scale (YBOCS), to assess the severity and frequency of the individual's symptoms.

In addition to the above, the individual may also be referred for a neuropsychological assessment to evaluate their cognitive functioning and rule out any underlying neurological conditions 

In [21]:
query = "When are you more likely to get a blood clot?"
response = rag_chain.invoke(query)
answer = response['text'].strip()
print("\nModel's Answer:\n")
print(answer)

/opt/conda/envs/pp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Model's Answer:

A blood clot occurs when blood clots together, forming a blockage in a blood vessel. Blood clots can occur in any part of the body, but they are most commonly found in the legs, arms, and lungs. There are several factors that can increase your risk of getting a blood clot. Here are some of the most common ones:

1. Age: As you age, your blood vessels become stiffer and less flexible, making it easier for blood clots to form.
2. Smoking: Smoking damages the lining of your blood vessels, making it easier for blood clots to form.
3. Obesity: Being overweight or obese can increase your risk of getting a blood clot, especially if you have high blood pressure or high cholesterol.
4. Sedentary lifestyle: Leading a sedentary lifestyle, such as sitting for long periods of time, can increase your risk of getting a blood clot.
5. Family history: If you have a family history of blood clots, you may be at higher risk of getting one yourself.
6. Medical conditions: Certain medical 

In [23]:
query = "How should you lift objects to prevent back pain?"
response = rag_chain.invoke(query)
answer = response['text'].strip()
print("\nModel's Answer:\n")
print(answer)

/opt/conda/envs/pp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Model's Answer:




In [25]:
query = "How can you be smart with antibiotics?"
response = rag_chain.invoke(query)
answer = response['text'].strip()
print("\nModel's Answer:\n")
print(answer)

/opt/conda/envs/pp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Model's Answer:

To be smart with antibiotics, it is important to follow the instructions provided by your healthcare provider and complete the full course of treatment as prescribed. Antibiotics are designed to kill specific types of bacteria, and taking them for too short a period of time or skipping doses can allow the remaining bacteria to multiply and become resistant to the antibiotic. Additionally, it is important to avoid taking antibiotics unnecessarily, as overuse can contribute to the development of antibiotic-resistant bacteria. Finally, it is important to practice good hygiene and take steps to prevent the spread of infection, such as washing your hands regularly and covering your mouth and nose when coughing or sneezing.
